# Mahoney Group ML Models - Model Registry

This notebook trains ML models for the Mahoney Group Intelligence Agent:
- **Claim Cost Prediction** - Predict total incurred costs for claims
- **High-Risk Claims Detection** - Classify claims as high-risk
- **Renewal Likelihood Prediction** - Predict policy renewal probability

All models are registered to Snowflake Model Registry and can be added as tools to the Intelligence Agent.

## Before You Begin

**Add these packages** in the Packages dropdown (upper right):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database:** MAHONEY_GROUP_INTELLIGENCE  
**Schema:** ANALYTICS  
**Warehouse:** MAHONEY_WH


## Import Required Packages


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake

Get active session and set context to Mahoney Group database.


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context
session.use_database('MAHONEY_GROUP_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('MAHONEY_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


---
# MODEL 1: Claim Cost Prediction

Predict total incurred costs for claims based on claim characteristics.

### Prepare Training Data


In [ ]:
# Get claim data with features for cost prediction
# Columns verified against 02_create_tables.sql lines 191-217, 16-36, 63-79, 84-105
claim_cost_df = session.sql("""
SELECT
    c.claim_id,
    c.claim_type,
    c.claim_category,
    c.loss_type,
    c.severity,
    cl.industry_vertical,
    cl.employee_count::FLOAT AS client_size,
    cl.risk_rating,
    prod.product_category,
    prod.coverage_type,
    p.policy_type,
    DATEDIFF('day', c.incident_date, c.report_date)::FLOAT AS days_to_report,
    c.litigation_involved::BOOLEAN AS has_litigation,
    c.subrogation_potential::BOOLEAN AS has_subrogation,
    c.claim_amount_incurred::FLOAT AS total_incurred
FROM RAW.CLAIMS c
JOIN RAW.CLIENTS cl ON c.client_id = cl.client_id
JOIN RAW.POLICIES p ON c.policy_id = p.policy_id
JOIN RAW.INSURANCE_PRODUCTS prod ON p.product_id = prod.product_id
WHERE c.claim_status = 'CLOSED'
  AND c.claim_amount_incurred > 0
  AND c.incident_date >= DATEADD('year', -3, CURRENT_DATE())
""")

print(f"Claim cost data: {claim_cost_df.count()} closed claims")
claim_cost_df.show(5)


### Train Claim Cost Model


In [ ]:
# Train/test split
train_cost, test_cost = claim_cost_df.random_split([0.8, 0.2], seed=42)

# Drop CLAIM_ID
train_cost = train_cost.drop("CLAIM_ID")
test_cost = test_cost.drop("CLAIM_ID")

# Create pipeline
cost_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["CLAIM_TYPE", "CLAIM_CATEGORY", "LOSS_TYPE", "SEVERITY", "INDUSTRY_VERTICAL", "RISK_RATING", "PRODUCT_CATEGORY", "COVERAGE_TYPE", "POLICY_TYPE"],
        output_cols=["CLAIM_TYPE_ENC", "CLAIM_CATEGORY_ENC", "LOSS_TYPE_ENC", "SEVERITY_ENC", "INDUSTRY_ENC", "RISK_ENC", "PRODUCT_CAT_ENC", "COVERAGE_ENC", "POLICY_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["CLIENT_SIZE", "DAYS_TO_REPORT"],
        output_cols=["CLIENT_SIZE_SCALED", "DAYS_TO_REPORT_SCALED"]
    )),
    ("Regressor", LinearRegression(
        label_cols=["TOTAL_INCURRED"],
        output_cols=["PREDICTED_COST"]
    ))
])

# Train model
cost_pipeline.fit(train_cost)
print("✅ Claim cost prediction model trained")


### Evaluate and Register Claim Cost Model


In [ ]:
# Make predictions on test set
cost_predictions = cost_pipeline.predict(test_cost)

# Calculate metrics
mae = mean_absolute_error(df=cost_predictions, y_true_col_names="TOTAL_INCURRED", y_pred_col_names="PREDICTED_COST")
mse = mean_squared_error(df=cost_predictions, y_true_col_names="TOTAL_INCURRED", y_pred_col_names="PREDICTED_COST")
rmse = mse ** 0.5

cost_metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Claim cost model metrics: {cost_metrics}")

# Register model to Model Registry
reg = Registry(session)
reg.log_model(
    model=cost_pipeline,
    model_name="CLAIM_COST_PREDICTOR",
    version_name="V1",
    comment="Predicts total incurred claim costs using Linear Regression based on claim type, client characteristics, and policy details",
    metrics=cost_metrics
)

print("✅ Claim cost model registered to Model Registry as CLAIM_COST_PREDICTOR")


---
# MODEL 2: High-Risk Claims Detection

Classify claims as high-risk based on characteristics and patterns indicating potential for high severity or complexity.


### Prepare High-Risk Claims Training Data


In [ ]:
# Get claim features for high-risk detection
# Columns verified against 02_create_tables.sql
highrisk_df = session.sql("""
SELECT
    c.claim_id,
    c.claim_type,
    c.claim_category,
    c.loss_type,
    c.severity,
    cl.industry_vertical,
    cl.risk_rating,
    prod.product_category,
    p.policy_type,
    DATEDIFF('day', c.incident_date, c.report_date)::FLOAT AS days_to_report,
    c.litigation_involved::BOOLEAN AS has_litigation,
    c.subrogation_potential::BOOLEAN AS has_subrogation,
    -- Target: High risk if severe, litigation, or high cost
    (c.severity = 'HIGH' 
     OR c.litigation_involved = TRUE 
     OR c.claim_amount_incurred > 50000 
     OR EXISTS (SELECT 1 FROM RAW.CLAIM_DISPUTES cd WHERE cd.claim_id = c.claim_id))::BOOLEAN AS is_high_risk
FROM RAW.CLAIMS c
JOIN RAW.CLIENTS cl ON c.client_id = cl.client_id
JOIN RAW.POLICIES p ON c.policy_id = p.policy_id
JOIN RAW.INSURANCE_PRODUCTS prod ON p.product_id = prod.product_id
WHERE c.claim_status IN ('CLOSED', 'SETTLED', 'PENDING')
  AND c.incident_date >= DATEADD('year', -3, CURRENT_DATE())
""")

print(f"High-risk claims data: {highrisk_df.count()} claims")
highrisk_df.show(5)


### Train High-Risk Claims Model


In [ ]:
# Train/test split
train_risk, test_risk = highrisk_df.random_split([0.8, 0.2], seed=42)

# Drop CLAIM_ID
train_risk = train_risk.drop("CLAIM_ID")
test_risk = test_risk.drop("CLAIM_ID")

# Create pipeline
risk_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["CLAIM_TYPE", "CLAIM_CATEGORY", "LOSS_TYPE", "SEVERITY", "INDUSTRY_VERTICAL", "RISK_RATING", "PRODUCT_CATEGORY", "POLICY_TYPE"],
        output_cols=["CLAIM_TYPE_ENC", "CLAIM_CATEGORY_ENC", "LOSS_TYPE_ENC", "SEVERITY_ENC", "INDUSTRY_ENC", "RISK_ENC", "PRODUCT_CAT_ENC", "POLICY_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_HIGH_RISK"],
        output_cols=["RISK_PREDICTION"],
        n_estimators=100,
        max_depth=15
    ))
])

# Train model
risk_pipeline.fit(train_risk)
print("✅ High-risk claims detection model trained")


### Evaluate and Register High-Risk Claims Model


In [ ]:
# Make predictions
risk_predictions = risk_pipeline.predict(test_risk)

# Calculate metrics
risk_accuracy = accuracy_score(df=risk_predictions, y_true_col_names="IS_HIGH_RISK", y_pred_col_names="RISK_PREDICTION")
risk_metrics = {"accuracy": round(risk_accuracy, 4)}
print(f"High-risk claims detection model metrics: {risk_metrics}")

# Register model
reg.log_model(
    model=risk_pipeline,
    model_name="HIGH_RISK_CLAIMS_DETECTOR",
    version_name="V1",
    comment="Classifies claims as high-risk using Random Forest based on severity, litigation, and claim patterns",
    metrics=risk_metrics
)

print("✅ High-risk claims model registered to Model Registry as HIGH_RISK_CLAIMS_DETECTOR")


---
# MODEL 3: Renewal Likelihood Prediction

Predict whether a client will renew their policy based on claims history, satisfaction, and relationship factors.


### Prepare Renewal Prediction Training Data


In [ ]:
# Get policy renewal data
# Columns verified against 02_create_tables.sql
renewal_df = session.sql("""
SELECT
    p.policy_id,
    cl.client_status,
    cl.industry_vertical,
    cl.business_segment,
    cl.risk_rating,
    cl.employee_count::FLOAT AS client_size,
    DATEDIFF('year', cl.onboarding_date, CURRENT_DATE())::FLOAT AS years_as_client,
    cl.client_satisfaction_score::FLOAT AS satisfaction_score,
    prod.product_category,
    prod.coverage_type,
    p.policy_type,
    DATEDIFF('year', p.effective_date, CURRENT_DATE())::FLOAT AS policy_years,
    (SELECT COUNT(*) FROM RAW.CLAIMS c WHERE c.policy_id = p.policy_id)::FLOAT AS claims_count,
    COALESCE((SELECT SUM(c.claim_amount_incurred) FROM RAW.CLAIMS c WHERE c.policy_id = p.policy_id), 0)::FLOAT AS total_claims_cost,
    CASE WHEN p.annual_premium > 0 
         THEN (COALESCE((SELECT SUM(c.claim_amount_incurred) FROM RAW.CLAIMS c WHERE c.policy_id = p.policy_id), 0) / p.annual_premium)::FLOAT
         ELSE 0.0 END AS loss_ratio,
    (EXISTS (SELECT 1 FROM RAW.POLICY_RENEWALS pr WHERE pr.policy_id = p.policy_id AND pr.renewal_status = 'RENEWED'))::BOOLEAN AS did_renew
FROM RAW.POLICIES p
JOIN RAW.CLIENTS cl ON p.client_id = cl.client_id
JOIN RAW.INSURANCE_PRODUCTS prod ON p.product_id = prod.product_id
WHERE p.policy_status IN ('EXPIRED', 'CANCELLED', 'ACTIVE')
  AND p.effective_date >= DATEADD('year', -5, CURRENT_DATE())
""")

print(f"Renewal prediction data: {renewal_df.count()} policies")
renewal_df.show(5)


### Train Renewal Likelihood Model


In [ ]:
# Train/test split
train_renewal, test_renewal = renewal_df.random_split([0.8, 0.2], seed=42)

# Drop POLICY_ID
train_renewal = train_renewal.drop("POLICY_ID")
test_renewal = test_renewal.drop("POLICY_ID")

# Create pipeline
renewal_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["CLIENT_STATUS", "INDUSTRY_VERTICAL", "BUSINESS_SEGMENT", "RISK_RATING", "PRODUCT_CATEGORY", "COVERAGE_TYPE", "POLICY_TYPE"],
        output_cols=["STATUS_ENC", "INDUSTRY_ENC", "SEGMENT_ENC", "RISK_ENC", "PRODUCT_CAT_ENC", "COVERAGE_ENC", "POLICY_TYPE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["CLIENT_SIZE", "YEARS_AS_CLIENT", "SATISFACTION_SCORE", "POLICY_YEARS", "CLAIMS_COUNT", "TOTAL_CLAIMS_COST", "LOSS_RATIO"],
        output_cols=["CLIENT_SIZE_SCALED", "YEARS_SCALED", "SATISFACTION_SCALED", "POLICY_YEARS_SCALED", "CLAIMS_COUNT_SCALED", "CLAIMS_COST_SCALED", "LOSS_RATIO_SCALED"]
    )),
    ("Classifier", LogisticRegression(
        label_cols=["DID_RENEW"],
        output_cols=["RENEWAL_PREDICTION"]
    ))
])

# Train
renewal_pipeline.fit(train_renewal)
print("✅ Renewal likelihood prediction model trained")


### Evaluate and Register Renewal Likelihood Model


In [ ]:
# Predict on test set
renewal_predictions = renewal_pipeline.predict(test_renewal)

# Calculate accuracy
renewal_accuracy = accuracy_score(df=renewal_predictions, y_true_col_names="DID_RENEW", y_pred_col_names="RENEWAL_PREDICTION")
renewal_metrics = {"accuracy": round(renewal_accuracy, 4)}
print(f"Renewal prediction model metrics: {renewal_metrics}")

# Register model
reg.log_model(
    model=renewal_pipeline,
    model_name="RENEWAL_LIKELIHOOD_PREDICTOR",
    version_name="V1",
    comment="Predicts policy renewal likelihood using Logistic Regression based on client satisfaction, claims history, and relationship factors",
    metrics=renewal_metrics
)

print("✅ Renewal prediction model registered to Model Registry as RENEWAL_LIKELIHOOD_PREDICTOR")


---
# Verify Models in Registry


In [ ]:
# Show all models in the registry
print("Models in registry:")
reg.show_models()

# Show versions for each model
print("\nClaim cost model versions:")
reg.get_model("CLAIM_COST_PREDICTOR").show_versions()

print("\nHigh-risk claims detector versions:")
reg.get_model("HIGH_RISK_CLAIMS_DETECTOR").show_versions()

print("\nRenewal likelihood predictor versions:")
reg.get_model("RENEWAL_LIKELIHOOD_PREDICTOR").show_versions()

print("\n✅ All models registered and ready to add to Intelligence Agent")


---
# Models Registered Successfully

All three models are now registered to the Snowflake Model Registry and ready to be called by the Intelligence Agent via the wrapper procedures in `sql/ml/07_create_model_wrapper_functions.sql`.


In [ ]:
# Verify all models are registered and accessible
print("✅ All models registered successfully:")
print("   1. CLAIM_COST_PREDICTOR")
print("   2. HIGH_RISK_CLAIMS_DETECTOR")
print("   3. RENEWAL_LIKELIHOOD_PREDICTOR")
print("\nModels ready to use via wrapper procedures in sql/ml/07_create_model_wrapper_functions.sql")


---
# Test Model Inference

Verify all models can be loaded from registry and make predictions.


In [ ]:
# Test claim cost prediction
cost_model = reg.get_model("CLAIM_COST_PREDICTOR").default
sample_cost = cost_df.limit(5).drop("CLAIM_ID")
cost_preds = cost_model.run(sample_cost, function_name="predict")
print("Claim cost predictions:")
cost_preds.select("CLAIM_TYPE", "SEVERITY", "TOTAL_INCURRED", "PREDICTED_COST").show()

# Test high-risk detection
risk_model = reg.get_model("HIGH_RISK_CLAIMS_DETECTOR").default
sample_risk = risk_df.limit(5).drop("CLAIM_ID")
risk_preds = risk_model.run(sample_risk, function_name="predict")
print("\nHigh-risk claim predictions:")
risk_preds.select("CLAIM_TYPE", "SEVERITY", "IS_HIGH_RISK", "HIGH_RISK_PREDICTION").show()

# Test renewal prediction
renewal_model = reg.get_model("RENEWAL_PREDICTOR").default
sample_renewal = renewal_df.limit(5).drop("POLICY_ID")
renewal_preds = renewal_model.run(sample_renewal, function_name="predict")
print("\nRenewal likelihood predictions:")
renewal_preds.select("PRODUCT_CATEGORY", "CLIENT_TENURE", "RENEWED", "RENEWAL_PREDICTION").show()

print("\n✅ All models tested successfully!")


---
# Next Steps

## Add Models to Intelligence Agent

1. Execute the wrapper procedures: `sql/ml/07_create_model_wrapper_functions.sql`
2. In Snowsight → AI & ML → Agents → MAHONEY_GROUP_INTELLIGENCE_AGENT
3. Go to Tools → + Add → Procedure
4. Add each procedure:
   - **PREDICT_CLAIM_COST**
   - **DETECT_HIGH_RISK_CLAIMS**
   - **PREDICT_RENEWAL_LIKELIHOOD**

## Example Questions for Agent

- "Predict claim costs for open property damage claims"
- "Which open claims are classified as high-risk according to the detector?"
- "Which clients are most likely to not renew their policies based on the renewal predictor?"
- "Analyze renewal likelihood for clients with high loss ratios"

The models are now available as tools your agent can use!
